In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [6]:
#Loading the csv data to a Pandas DataFrame
heart_data = pd.read_csv("heart.csv")

In [15]:
#Printing the first and last five rows of Dataset
heart_data.head()
#heart_data.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


We need to predict the values for "target" column which tells
whether a person has Heart Disease or Not.

In [7]:
# Get no of rows and columns in the dataset
heart_data.shape

(1025, 14)

In [18]:
#Getting information about Dataset
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [8]:
#Another way for checking the missing values in Dataset
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [9]:
#Statistical measures about the data
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [10]:
#Determining How many people has Heart Disease
has_disease = heart_data[heart_data.target==1]
print(f"People having Heart Disease : {has_disease.shape[0]}")
not_having_disease = heart_data[heart_data.target==0]
print(f"People not having Heart Disease : {not_having_disease.shape[0]}")

People having Heart Disease : 526
People not having Heart Disease : 499


In [11]:
#Splitting the Features and Target Column
X = heart_data.drop(columns="target",axis=1)
Y = heart_data["target"]

In [12]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2


In [13]:
Y

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1025, dtype: int64

In [14]:
#Splitting the Data into Training Data and Testing Data

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, stratify = Y,random_state = 2)

In [15]:
#Check the Training and Testing Data Shape
print(f"Training Data : {x_train.shape}\nTesting Data : {x_test.shape}")

Training Data : (820, 13)
Testing Data : (205, 13)


In [35]:
#Selecting and Predicting the Best Model for Prediction

In [16]:
#Initialise the models

models = {
    "Random Forest" : RandomForestClassifier(),
    "Support Vector Machine" : SVC(),
    "Logistic Regression" : LogisticRegression(max_iter=1000)
}

In [17]:
#Getting all the parameters of our Model

print(f"Parameters for Logistic Regression : {models['Logistic Regression'].get_params()}")
print("-"*70)
print(f"Parameters for Support Vector Machine : {models['Support Vector Machine'].get_params()}")
print("-"*70)
print(f"Parameters for Random Forest : {models['Random Forest'].get_params()}")
print("-"*70)

Parameters for Logistic Regression : {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
----------------------------------------------------------------------
Parameters for Support Vector Machine : {'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
----------------------------------------------------------------------
Parameters for Random Forest : {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease'

In [57]:
#Perform Cross-Validation and Tune HyperParameters

params_grid = {
    "Random Forest" : {
        'n_estimators' : [50,100,150],
        'max_depth' : [10,20,None],
        'min_samples_split' : [2,5,10]
    },
    "Support Vector Machine" : {
        'C' : [0.1,1,10],
        'kernel' : ["linear","rbf"]
    },
    "Logistic Regression" : {
        'C' : [0.1,1,10],
        'solver' : ["liblinear","lbfgs"]
    }
}

best_models = {}
best_score = {}

for model_name, model in models.items():
    print(f"Tuning the hyper-parameters for {model_name}:-")
    grid = GridSearchCV(estimator=model,param_grid=params_grid[model_name], scoring="accuracy", cv=5, n_jobs = -1)
    grid.fit(x_train, y_train)

    best_models[model_name] = grid.best_estimator_
    best_score[model_name] = grid.best_score_
    print(f"Best parameters for {model_name} : {grid.best_params_}")
    print(f"Best Cross-Validation score for {model_name} : {grid.best_score_}\n")

best_model_name = max(best_score, key=best_score.get)
best_model = best_models[best_model_name]

print(f"Best Model : {best_model_name}")

best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)

Tuning the hyper-parameters for Random Forest:-
Best parameters for Random Forest : {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation score for Random Forest : 0.9817073170731707

Tuning the hyper-parameters for Support Vector Machine:-
Best parameters for Support Vector Machine : {'C': 1, 'kernel': 'linear'}
Best Cross-Validation score for Support Vector Machine : 0.8609756097560975

Tuning the hyper-parameters for Logistic Regression:-
Best parameters for Logistic Regression : {'C': 10, 'solver': 'lbfgs'}
Best Cross-Validation score for Logistic Regression : 0.852439024390244

Best Model : Random Forest


In [63]:
#Print results

print(f"Accuracy on Test set : {accuracy_score(y_test, y_pred)}\nClassification Report : \n{classification_report(y_test,y_pred)}\nConfusion Matrix : \n{confusion_matrix(y_test, y_pred)}")
print("Accuracy Score : ")

Accuracy on Test set : 1.0
Classification Report : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       105

    accuracy                           1.00       205
   macro avg       1.00      1.00      1.00       205
weighted avg       1.00      1.00      1.00       205

Confusion Matrix : 
[[100   0]
 [  0 105]]


In [71]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*X does not have valid feature names.*")

#Building a Predictive system

input_data = (43,0,0,132,341,1,0,136,1,3,1,0,3)

#Changing input data as numpy array
input_array = np.asarray(input_data)

# reshape the numpy array as we are predicting for only 1 instance

input_array_reshaped = input_array.reshape(1,-1)

pred_value = best_model.predict(input_array_reshaped)
print(pred_value) #List is returned

if pred_value[0] == 0:
    print("Person doesnot have any Heart Disease")
else:
    print("Person has Heart Disease")

[0]
Person doesnot have any Heart Disease
